In [ ]:
import os
import sys
sys.path.append('../release')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from collections import defaultdict
import rdkit.Chem as Chem
from rdkit.Chem import Descriptors
from sklearn.ensemble import RandomForestClassifier as RFC

In [ ]:
from analysis_utils import LogMiner, get_fingerprint_similarities, get_property, plot_scaffolds

In [ ]:
log_dir = '../logs/'

# Generate barchart with summary metrics

In [ ]:
# extract data from logs
name = 'replay_combo'
dates = (201111, 201112, 201113)

for date in dates:
    log_path = os.path.join(log_dir, name + str(date) + '.log')
    lms.append(LogMiner(log_path, 3))  # smooth data with a moving average of 3
results = pd.concat([lm.results().iloc[[6,7,3,4]] for lm in lms])
results['params'] = results.params.apply(lambda d: tuple(d.items()))
results = results.groupby('params')

In [ ]:
# plot barchart
cmap = plt.get_cmap('coolwarm')
colors = cmap([192, 64])
width = 0.3
x = np.arange(len(results))
plt.figure(dpi=300)
ax1 = plt.axes()
for i, field in enumerate(('active_fraction', 'valid_fraction')):
    label = field.replace('_', ' ')
    label = label[:1].upper() + label[1:]
    ax1.bar(x + (i-0.5)*width, results.mean()[field].iloc[[2,3,0,1]], width,
            yerr=results.sem()[field].iloc[[2,3,0,1]], label=label, capsize=3, color=colors[i])
    plt.legend()
    plt.xlim(-0.5, len(results) - 0.5)
    plt.ylabel('Active / Valid fraction')

rows = ['', 'Fine-tune', 'Replay']
cell_text = [[' ', ' ', ' ', ' '],
             ['\u2212', '+', '\u2212', '+'],
             ['\u2212', '\u2212', '+', '+']]
plt.table(cellText=cell_text,
          rowLabels=rows,
          rowLoc='right',
          cellLoc='center',
          edges='R',
          loc='bottom')
plt.xticks([])
plt.show()

# Plot most important fields over training

In [ ]:
# extract run data
name = 'replay_ratio'
dates = (201212, 201213, 201214)

for date in dates:
    log_path = os.path.join(log_dir, name + str(date) + '.log')
    lms.append(LogMiner(log_path, 3))

In [ ]:
# collate run data by parameter
tot_runs = defaultdict(list)
for lm in lms:
    for run in lm.runs():
        # add data for epoch 0
        run.data = pd.concat((run.data.iloc[0:1], run.data))
        run.data['actives_fine_tune'].iloc[0] = run.data.actives_policy.iloc[0]
        run.data['valids_fine_tune'].iloc[0] = run.data.valids_policy.iloc[0]
        run.data['epoch'].iloc[0] = 0
        params = tuple(run.params.items())
        tot_runs[params].append(run)

In [ ]:
# plot metrics over training
n = len(tot_runs)
cmap = plt.get_cmap('winter', n)
colors = cmap(range(n))
fig, axes = plt.figure(figsize=(4, 4), sharex=True)
axes = axes.flatten()
labels = []
for i, (params, runs) in enumerate(tot_runs.items()):
    params = dict(params)
    label = '%d / %d' % (params['n_policy'], params['n_policy_replay'])
    labels.append(label)
    results = (pd.concat([run.data for run in runs])
              .groupby('epoch'))
    epochs = list(results.groups)
    for j, field in enumerate(('actives_fine_tune',
                               'valids_fine_tune',
                               'thresholds',
                               'rewards')):
        axes[j].errorbar(epochs, results[field].mean(), yerr=results[field].sem(),
                         colors=colors[i], elinewidth=0.1, capsize=0.2)
        upper = results[field].mean() + results[field].sem()
        lower = results[field].mean() - results[field].sem()
        axes[j].fill_between(epochs, lower, upper, colors[i], alpha=0.1)

for ax, field in zip(axes, ('Active fraction',
                            'Valid fraction',
                            'Replay threshold',
                            'Rewards')):
    ax.set_title(field)

axes[2].set_xlabel('Epoch')
axes[2].set_ybound(lower=-0.05)
axes[3].set_xlabel('Epoch')
axes[3].set_ybound(lower=-0.5)
# invisible plot to add colorbar
data = axes[3].scatter([2]*n, [0]*n, s=0, c=np.arange(n), cmap=cmap)
data.set_clim(0, n)
cbar = plt.colorbar(data, ax=axes, location='bottom', pad=0.2)
cbar.set_ticks(np.arange(n) + 0.5)
cbar.ax.set_xticklabels(labels)
plt.show()

In [ ]:
# plot trajectory of active and valid fractions over training
n = len(tot_runs)
cmap = plt.get_cmap('winter', n)
colors = cmap(range(n))
plt.figure(figsize=(4,4))
for i, (params, runs) in enumerate(tot_runs.items()):
    params = dict(params)
    label = '%d / %d' % (params['n_policy'], params['n_policy_replay'])
    labels.append(label)
    for j, run in enumerate(runs):
        x = run.data['actives_fine_tune']
        y = run.data['valids_fine_tune']
        plt.plot(x, y, '.-', alpha=0.25, color=colors[i])
        plt.scatter(x.iloc[-1], y.iloc[-1], color=colors[i], zorder=4)
plt.xlabel('Active fraction')
plt.ylabel('Valid fraction')
plt.title('Valid fraction versus Active fraction')
data = plt.scatter([0.5]*n, [0.5]*n, s=0, c=np.arange(n), cmap=cmap)
data.set_clim(0, n)
cbar = plt.colorbar(data, location='bottom', pad=0.2)
cbar.set_ticks(np.arange(n) + 0.5)
cbar.ax.set_xticklabels(labels)
plt.show()

# Plot library distributions

In [ ]:
# violinplot with option to pass colormaps
def violinplot(dataset, c=None, cmap=None, **kwargs):
    n = len(dataset)
    if c is not None:
        c = plt.get_cmap(cmap, n)(c)
    
    plot = plt.violinplot(dataset, **kwargs)
    for key in ('cmins', 'cmaxes', 'cbars'):
        plot[key].set_linewidth(0.5)
    for val in plot.values():
        if isinstance(val, list):
            for i in range(n):
                val[i].set_color(c[i])
        else:
            val.set_array(np.linspace(0,1,n))
            val.set_cmap(cmap)
    return plot

### Plot predicted activity distribution of timelapse data

In [ ]:
name = 'timelapse201106'

log_path = os.path.join(log_dir, name)
n = len(glob(log_path + '*.smi'))
libs = []
for i in range(n):
    tmp = pd.read_csv(log_path + '-%d.smi' % (2*i), names=['smiles', 'predictions'])
    tmp['molecules'] = tmp.smiles.apply(Chem.MolFromSmiles)
    libs.append(tmp)
mols = [lib.molecules for lib in libs]

In [ ]:
t = 2 * np.arange(n)
cmap = plt.get_cmap('cool', n)
colors = cmap(range(n))
fig, ax = plt.subplots(figsize=9, 4.5)
predictions = [lib.predictions for lib in libs]
means = np.array(p.mean() for p in predictions)
violinplot(predictions, c=np.arange(n), cmap='cool',
           positions=t, widths=2, bw_method=0.1)
plt.plot(t, means, 'b-')
plt.xlabel('Epochs trained')
plt.xticks([0,4,8,12,16,20])
plt.ylabel('Predicted activity')
plt.title('Distribution of predicted activities over training')
plt.show()

### Plot distribution of similarities of generated libraries relative to experimental actives

In [ ]:
# load generated libraries
name = 'replay_data201104'

log_path = log_dir + name
n_runs = len(glob(log_path + '*.smi'))
n = n_runs - 1
conditions = ['Empty buffer', 'Generated actives', 'Enamine']
for i in range(n):
    tmp = pd.read_csv(log_path + '-%d.smi' % i, names=['smiles', 'predictions'])
    tmp['molecules'] = tmp.smiles.apply(Chem.MolFromSmiles)
    libs.append(tmp)
mols = [lib.molecules for lib in libs]

In [ ]:
# load experimental library as a baseline
egfr_actives = pd.read_csv('../data/egfr_actives.smi', names=['smiles', 'predictions'])
egfr_actives['molecules'] = egfr_actives.smiles.apply(Chem.MolFromSmiles)
egfr_actives = egfr_actives[egfr_actives.predictions > 0.75]

In [ ]:
# calculate similarity distributions of libraries
sims = {}
sim = get_fingerprint_similarities(data, sample_size=1000)

max_sim = [max(l) for l in sim]
sims['Max Similarity'] = max_sim
sims['Similarity'] = sim

In [ ]:
t = np.arange(n)
cmap = plt.get_cmap('brg', n)
colors = cmap(t)
fig, axes = plt.subplots(2, 1, figsize=(6,6), sharex=True)
axes = axes.flatten()
for i, (prop, dist) in enumerate(sims.items()):
    medians = np.array([np.median(d) for d in dist])
    q1s = np.array([np.percentile(d, 25) for d in dist])
    q3s = np.array([np.percentile(d, 75) for d in dist])
    plt.sca(axes[i])
    violinplot(dist, c=t, cmap='brg',
               positions=t, widths=1, bw_method=0.3)
    for j in range(n):
        plt.plot(t[j], medians[j], 'o', c=colors[j])
        plt.plot([t[j], t[j]], [q1s[j], q3s[j]], c=colors[j], lw=2.5)
    plt.ylabel('Similarity')
    plt.title('Distribution of %s' % prop)
plt.xticks(t)
plt.gca().set_xticklabels(['Experimental actives',
                           'Empty buffer',
                           'Generated actives',
                           'Enamine'])
plt.show()

# Display sample scaffolds of various runs

In [ ]:
name = 'replay_data201104'

log_path = path + name
n_runs = len(glob(log_path + '*.smi'))
n = n_runs - 1
conditions = ['empty buffer', 'generated actives', 'enamine']

In [ ]:
n_cols = 4
n_cols = min(n_cols, n)
n_rows = (n-1) // n_cols + 1
n_to_show = 12

fig, axes = plt.subplots(1, 3, dpi=300)
axes = axes.flatten()
libs = []
for i in range(n_runs - 1):
    lib_path = log_path + '-%d.smi' % (i+1)
    mol_data = pd.read_csv(lib_path, names=['smiles', 'predictions'])
    mol_data = mol_data[mol_data.predictions > 0.75]
    mol_data['molecules'] = mol_data.smiles.apply(Chem.MolFromSmiles)
    libs.append(mol_data['molecules'])
for i in range(len(libs)):
    plot_scaffolds(libs[i], ax=axes[i], subImgSize=(1000,1000), useSVG=True)
    axes[i].set_title(conditions[i])
    axes[i].axis('off')
plt.show()